In [12]:
import pandas as pd

#google cloud big query libaries
from google.cloud import bigquery
from google.oauth2 import service_account

import os

In [13]:
dirShiny = "E:/GitHub/Resources/R-Shiny/HTS-Trip-Lengths/input"

group_by_columns = ['groupSampleSegment','groupNumWorkers','groupNumVehicles','groupTripType','groupModeTypeBroad']
display(group_by_columns)

['groupSampleSegment',
 'groupNumWorkers',
 'groupNumVehicles',
 'groupTripType',
 'groupModeTypeBroad']

# Setup Data

In [14]:
# Link to BigQuery Client through API

key_path = r"C:\Users\bhereth\confidential-2023-utah-hts-db5335615978.json"
#key_path = r"C:\Users\bhereth\tdm-scenarios-a85044dbbfd3.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
print ('Successfully Linked to BigQuery Client!')

Successfully Linked to BigQuery Client!


In [15]:
str_sql_trips = """
SELECT
  gSs.groupSampleSegment,
  gNw.groupNumWorkers,
  gNv.groupNumVehicles,
  gMtb.groupModeTypeBroad,
  trip.trip_type AS trip_type,
  COUNT(*) as numTripRecords,
  ROUND(SUM(trip.trip_weight),2) AS sumTripWeight
FROM
  `confidential-2023-utah-hts.20230313.hh` as hh,
  `confidential-2023-utah-hts.20230313.trip` as trip,
  `confidential-2023-utah-hts.20230313_groupings.groupSampleSegment` as gSs,
  `confidential-2023-utah-hts.20230313_groupings.groupNumWorkers` as gNw,
  `confidential-2023-utah-hts.20230313_groupings.groupNumVehicles` as gNv,
  `confidential-2023-utah-hts.20230313_groupings.groupModeTypeBroad` as gMtb
WHERE
  hh.sample_segment = gSs.sample_segment AND
  hh.hh_id = trip.hh_id AND
  hh.num_workers = gNw.num_workers AND
  hh.num_vehicles = gNv.num_vehicles AND
  trip.mode_type_broad = gMtb.mode_type_broad
GROUP BY
  gSs.groupSampleSegment,
  gNw.groupNumWorkers,
  gNv.groupNumVehicles,
  gMtb.groupModeTypeBroad,
  trip.trip_type
"""

In [16]:
str_sql_hh = """
SELECT
  gSs.groupSampleSegment,
  gNw.groupNumWorkers,
  gNv.groupNumVehicles,
  COUNT(*) as numHhRecords,
  ROUND(SUM(hh.hh_weight),2) AS sumHhWeight
FROM
  `confidential-2023-utah-hts.20230313.hh` as hh,
  `confidential-2023-utah-hts.20230313_groupings.groupSampleSegment` as gSs,
  `confidential-2023-utah-hts.20230313_groupings.groupNumWorkers` as gNw,
  `confidential-2023-utah-hts.20230313_groupings.groupNumVehicles` as gNv
WHERE
  hh.sample_segment = gSs.sample_segment AND
  hh.num_workers = gNw.num_workers AND
  hh.num_vehicles = gNv.num_vehicles
GROUP BY
  gSs.groupSampleSegment,
  gNw.groupNumWorkers,
  gNv.groupNumVehicles
"""

In [17]:
# test connection
df_trips_by_type = client.query(str_sql_trips).to_dataframe()

display(df_trips_by_type)

,groupSampleSegment,groupNumWorkers,groupNumVehicles,groupModeTypeBroad,trip_type,numTripRecords,sumTripWeight
0,WF,-1,3,5,6,108,4703.52
1,WF,-1,3,-1,6,3146,220844.47
2,WF,-1,-1,5,6,466,19814.33
3,WF,-1,-1,-1,6,19590,925526.43
4,WF,3,3,5,6,75,977.76
...,...,...,...,...,...,...,...
24676,Bo,3,6,995,995,2,16.76
24677,Un,4,1,995,995,3,14.51
24678,WF,3,6,995,995,2,1263.43
24679,Sa,3,6,995,995,2,1263.43


In [18]:
df_trips_by_type[df_trips_by_type['trip_type']==995]

,groupSampleSegment,groupNumWorkers,groupNumVehicles,groupModeTypeBroad,trip_type,numTripRecords,sumTripWeight
24,WF,-1,5,5,995,13,820.84
25,WF,-1,5,-1,995,1006,51159.65
26,WF,-1,-1,5,995,710,37972.51
27,WF,-1,-1,-1,995,57659,1941406.20
28,WF,3,5,5,995,2,96.11
...,...,...,...,...,...,...,...
24676,Bo,3,6,995,995,2,16.76
24677,Un,4,1,995,995,3,14.51
24678,WF,3,6,995,995,2,1263.43
24679,Sa,3,6,995,995,2,1263.43


In [19]:
# test connection
df_hh = client.query(str_sql_hh).to_dataframe()

display(df_hh)

,groupSampleSegment,groupNumWorkers,groupNumVehicles,numHhRecords,sumHhWeight
0,TT,-1,-1,11183,1137207.98
1,TT,-1,4,491,70403.31
2,TT,2,-1,3601,338730.63
3,TT,2,4,149,22666.22
4,Bo,-1,-1,449,46262.44
...,...,...,...,...,...
763,Su,3,0,1,14.98
764,TT,7,4,1,16.25
765,Su,7,-1,1,16.25
766,Su,7,4,1,16.25


In [20]:
df_trips_by_type_hh = pd.merge(df_trips_by_type, df_hh, on=[col for col in group_by_columns if col not in ['groupTripType', 'groupModeTypeBroad']])
df_trips_by_type_hh['tripsPerHh'] = df_trips_by_type_hh['sumTripWeight'] / df_trips_by_type_hh['sumHhWeight'] 
df_trips_by_type_hh['tripsPerHh'].fillna(0, inplace=True)
df_trips_by_type_hh

,groupSampleSegment,groupNumWorkers,groupNumVehicles,groupModeTypeBroad,trip_type,numTripRecords,sumTripWeight,numHhRecords,sumHhWeight,tripsPerHh
0,WF,-1,3,5,6,108,4703.52,906,139950.11,0.033609
1,WF,-1,3,-1,6,3146,220844.47,906,139950.11,1.578023
2,WF,-1,3,1,7,790,32005.84,906,139950.11,0.228695
3,WF,-1,3,-1,7,9751,361858.61,906,139950.11,2.585626
4,WF,-1,3,1001,7,883,32469.49,906,139950.11,0.232008
...,...,...,...,...,...,...,...,...,...,...
24676,Al,3,1,3,2,1,59.59,1,36.31,1.641146
24677,Al,3,1,-1,2,1,59.59,1,36.31,1.641146
24678,Al,3,1,3,7,1,36.31,1,36.31,1.000000
24679,Al,3,1,-1,7,1,36.31,1,36.31,1.000000


In [25]:
df_trip_types = client.query("SELECT * FROM confidential-2023-utah-hts.20230313.value_labels WHERE table='trip' AND variable='trip_type'").to_dataframe()
df_trip_types.drop(columns=['table','variable'], inplace=True)
df_trip_types.rename(columns={'value':'trip_type','label':'tripTypeLabel'}, inplace=True)
df_trip_types['tripTypeLabel'] = df_trip_types['tripTypeLabel'].apply(lambda x: '\n'.join(x.split()))
df_trip_types['tripTypeLabel'] = df_trip_types['tripTypeLabel'].apply(lambda x: '-\n'.join(x.split('-')))
df_trip_types['trip_type'] = df_trip_types['trip_type'].astype(int)
df_trip_types

,trip_type,tripTypeLabel
0,4,Home-\nbased\npersonal\nbusiness
1,2,Home-\nbased\nschool
2,6,Non-\nhome-\nbased\nwork
3,7,Non-\nhome-\nbased\nnon-\nwork
4,1,Home-\nbased\nwork
5,3,Home-\nbased\nshopping
6,5,Home-\nbased\nother
7,995,Missing\nresponse


In [27]:
df_trips_by_type_hh_with_labels = pd.merge(df_trips_by_type_hh, df_trip_types, on='trip_type')
df_trips_by_type_hh_with_labels

,groupSampleSegment,groupNumWorkers,groupNumVehicles,groupModeTypeBroad,trip_type,numTripRecords,sumTripWeight,numHhRecords,sumHhWeight,tripsPerHh,tripTypeLabel
0,WF,-1,3,5,6,108,4703.52,906,139950.11,0.033609,Non-\nhome-\nbased\nwork
1,WF,-1,3,-1,6,3146,220844.47,906,139950.11,1.578023,Non-\nhome-\nbased\nwork
2,WF,-1,3,1,6,237,14026.42,906,139950.11,0.100224,Non-\nhome-\nbased\nwork
3,WF,-1,3,1001,6,289,16058.73,906,139950.11,0.114746,Non-\nhome-\nbased\nwork
4,WF,-1,3,3,6,2579,195394.25,906,139950.11,1.396171,Non-\nhome-\nbased\nwork
...,...,...,...,...,...,...,...,...,...,...,...
24676,Ir,4,5,-1,1,2,34.68,1,10.57,3.280984,Home-\nbased\nwork
24677,Ir,3,5,3,1,1,93.07,1,56.71,1.641157,Home-\nbased\nwork
24678,Ir,3,5,-1,1,1,93.07,1,56.71,1.641157,Home-\nbased\nwork
24679,Ir,1,5,3,1,1,16.80,1,10.24,1.640625,Home-\nbased\nwork


In [28]:
df_trips_by_type_hh_with_labels.to_csv('trips_by_type.csv', index=False)